### * 品類重分類

In [166]:
import pandas as pd
import numpy as np
import jieba

#### 1. 斷詞

In [ ]:
usecols = ['customer_id', 'inovice_no', 'item_name', 'Item_category']

In [ ]:
invoice = pd.read_excel('./invoice_20241113.xlsx', usecols=usecols)

In [ ]:
cat_dict = invoice.groupby('Item_category', dropna=False)['item_name'].apply(list).to_dict()

In [ ]:
cat_dict_len = {k: len(v) for k, v in cat_dict.items()}
print(sum(cat_dict_len.values()))

In [ ]:
stopwords = set()

with open('./stop_words.txt', 'r', encoding='utf-8') as file:
  for line in file:
    stopwords.add(line.strip())

##### 1-1. 關鍵字

In [ ]:
keywords_dict = {}
keywords_weight_dict = {}

for k, v in cat_dict.items():
  v = ' '.join(v)
  keywords = jieba.analyse.extract_tags(v, topK=50, withWeight=False, allowPOS=()) # allowPOS 指定詞性
  keywords_weight = jieba.analyse.extract_tags(v, topK=50, withWeight=True, allowPOS=())
  keywords_dict[k] = keywords
  keywords_weight_dict[k] = keywords_weight

In [ ]:
keywords_df = pd.DataFrame(keywords_dict)

##### 1-2. 最常出現的字

In [ ]:
from collections import Counter

In [ ]:
common_dict = {}

for k, v in cat_dict.items():
  v = ' '.join(v)
  common = jieba.lcut(v)
  common = [word for word in common if (word not in stopwords) and (len(word) > 1) and not any(i.isdigit() for i in word)]
  counter = Counter(common)
  most_common = counter.most_common(50)
  most_common_words = [w[0] for w in most_common]
  common_dict[k] = most_common_words

In [ ]:
common_df = pd.DataFrame(common_dict)

##### 1-3. 輸出

In [ ]:
with pd.ExcelWriter('./keywords.xlsx') as writer:
  common_df.to_excel(writer, sheet_name='most_common', index=False)

#### 2. 重分類

In [167]:
invoice = pd.read_excel("./invoice_20241113.xlsx")

In [168]:
invoice_copy = invoice.copy()

In [169]:
usecols = ['Item Name', '原品類', '新品類']

In [170]:
cat_df = pd.read_excel("./1125 任務盤點.xlsx", sheet_name="資料髒值處理", usecols=usecols)

In [171]:
cat_df.loc[:, '原品類'] = cat_df['原品類'].apply(lambda x: np.nan if x == '無品類' else x)

In [172]:
cat_dict = {}

In [173]:
for idx, row in cat_df.iterrows():
    if row['原品類'] not in cat_dict.keys():
        cat_dict[row['原品類']] = []
    cat_dict[row['原品類']].append((row['Item Name'], row['新品類']))

In [174]:
for k, v in cat_dict.items():
    try:
        if np.isnan(k):
            for s in v:
                k_index = invoice[(invoice['Item_category'].isna()) & (invoice['item_name'].str.contains(s[0].strip()))].index
                invoice_copy.loc[k_index, 'Item_category'] = s[1]
    except:
        for s in v:
            k_index = invoice[(invoice['Item_category'] == k) & (invoice['item_name'].str.contains(s[0].strip()))].index
            invoice_copy.loc[k_index, 'Item_category'] = s[1]

In [175]:
channel_dict = {'成人‧情趣': [('宜家家居', '生活日用品'), ('韓商美迪特拉比', '美妝‧保養')]}

In [176]:
for k, v in channel_dict.items():
    for s in v:
        k_index = invoice[(invoice['Item_category'] == k) & (invoice['seller_name_adjusted'] == s[0].strip())].index
        invoice_copy.loc[k_index, 'Item_category'] = s[1]

In [181]:
invoice_copy.to_excel("./invoice_20241126.xlsx", index=False)